Since the fancy clustering wasn't computationally efficent enough for the size of our data, we just use the built-in RDKit picking framework.

In [1]:
import pandas as pd
import rdkit
from rdkit import DataStructs
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprint
from rdkit.SimDivFilters.rdSimDivPickers import MaxMinPicker

In [2]:
df = pd.read_csv("../data/interim/negative_smiles.csv")

In [3]:
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,SMILES
0,CCOCCCNCC(=O)NC1=CC=C(C=C1)OC(F)(F)F.Cl
1,COCCN1C(=NN=N1)CN2CCC(CC2)CC3=CC=CC=C3.Cl
2,COCCN1C(=NN=N1)CN2CCC(CC2)(C3=CC(=CC=C3)C(F)(F...
3,C1CCCN(CC1)CC(=O)NCCC2=CC=C(C=C2)F.C(=O)(C(=O)O)O
4,COC1=CC=C(C=C1)C(=O)C(C2=CC=CC=C2)N3CCOCC3.Cl


In [4]:
ms = [rdkit.Chem.MolFromSmiles(x) for x in df["SMILES"]]

RDKit WARNING: [12:59:46] WARNING: not removing hydrogen atom without neighbors
[12:59:46] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [12:59:46] WARNING: not removing hydrogen atom without neighbors
[12:59:46] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [13:00:10] WARNING: not removing hydrogen atom without neighbors
[13:00:10] WARNING: not removing hydrogen atom without neighbors


In [5]:
fps = [GetMorganFingerprint(x,3) for x in ms]

In [6]:
def distij(i,j,fps=fps):
    return 1-DataStructs.DiceSimilarity(fps[i],fps[j])

picker = MaxMinPicker()
pickIndices = picker.LazyPick(distij,len(fps),2000,seed=23)

In [7]:
picks = [df["SMILES"][x] for x in pickIndices]

In [10]:
with open("../data/interim/negative_smiles_subset.csv", 'w') as f:
    f.write("SMILES\n")
    for i in picks:
        if i:
            f.write("%s\n" % i)